In [1]:
# data augmentation test: rotate different degree (pay attention to adjustable filename etc.)

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import layers, Sequential,losses, metrics

image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness',
                  'anger', 'disgust', 'fear', 'contempt']

# !!! change sample size
samples = 130967 # 2~130968
training_samples = 28317 *4  # 2~113269 (Training)
validation_samples = 3541 *4 # 113270~127433 (PublicTest)
test_samples = 3535         # 127434~130968 (PrivateTest)

# !!! change npy folder name
image_path = "./dataset3/images.npy"
emotion_multi_path = "./dataset3/emotions_multi.npy"
emotion_single_path = "./dataset3/emotions_single.npy"
images = np.load(image_path)
emotions_multi = np.load(emotion_multi_path)
emotions_single = np.load(emotion_single_path)

# !!! change s/m dataset
#emotions = emotions_single
emotions = emotions_multi

print(images.shape)
print(emotions_multi.shape)
print(emotions_single.shape)

(130967, 48, 48, 1)
(130967, 8)
(130967, 8)


In [2]:
cce = losses.CategoricalCrossentropy()
mse = losses.MeanSquaredError()

tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [3]:
images = tf.convert_to_tensor(images)

emotions = tf.convert_to_tensor(emotions)

images = layers.Rescaling(1./127.5, offset= -1)(images)

training_size = training_samples + validation_samples
test_size = test_samples

training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

print("training_images shape:", training_images.shape)
print("training_emotions shape:", training_emotions.shape)
print("test_images shape:", test_images.shape)
print("test_emotions shape:", test_emotions.shape)

training_images shape: (127432, 48, 48, 1)
training_emotions shape: (127432, 8)
test_images shape: (3535, 48, 48, 1)
test_emotions shape: (3535, 8)


In [4]:
from tensorflow.python.keras.applications import vgg16, resnet_v2
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.optimizer_v2 import adam
import matplotlib.pyplot as plt

In [5]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    model = Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

    model.compile(optimizer=adam.Adam(learning_rate=1e-4), 
                  loss=mse, 
                  metrics = [model_acc])
    return model

model = create_model()

best_model_save_path = "best_models/FERPlus_flip_+-25_multi_mse"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=best_model_save_path,
    save_weights_only=False, 
    monitor='val_model_acc',
    mode='max',
    save_best_only=True)

history = model.fit(x=tf.image.grayscale_to_rgb(training_images),
          y=training_emotions,
          batch_size=32,
          epochs=40,
          validation_data=(tf.image.grayscale_to_rgb(test_images), test_emotions),
          callbacks=[model_checkpoint_callback])

C:\Users\Darkl\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/40
3983/3983 [==============================] - 264s 66ms/step - loss: 0.0243 - model_acc: 0.7277 - val_loss: 0.0165 - val_model_acc: 0.7985
INFO:tensorflow:Assets written to: best_models\FERPlus_flip_+-25_multi_mse\assets
Epoch 2/40
3983/3983 [==============================] - 263s 66ms/step - loss: 0.0148 - model_acc: 0.8192 - val_loss: 0.0155 - val_model_acc: 0.8143
INFO:tensorflow:Assets written to: best_models\FERPlus_flip_+-25_multi_mse\assets
Epoch 3/40
3983/3983 [==============================] - 260s 65ms/step - loss: 0.0116 - model_acc: 0.8531 - val_loss: 0.0133 - val_model_acc: 0.8366
INFO:tensorflow:Assets written to: best_models\FERPlus_flip_+-25_multi_mse\assets
Epoch 4/40
3983/3983 [==============================] - 259s 65ms/step - loss: 0.0093 - model_acc: 0.8797 - val_loss: 0.0123 - val_model_acc: 0.8397
INFO:tensorflow:Assets written to: best_models\FERPlus_flip_+-25_multi_mse\assets
Epoch 5/40
3983/3983 [==============================] - 258s 65ms/step - los

In [7]:
best_model_path = "best_models/FERPlus_flip_+-25_multi_mse"
best_model = tf.keras.models.load_model(best_model_path, custom_objects={'model_acc': model_acc})
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 4096)              2101248   
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 32776     
Total params: 33,630,024
Trainable params: 33,630,024
Non-trainable params: 0
_________________________________________________________________
